In [ ]:
#instalamos la libreria STREAMLIT
%pip install streamlit

In [3]:
#Instalamos la libreria de PLOTLY
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Instalamos libreria para análisis de variables categóricas
%pip install funpymodeling

In [5]:
#Para instalar npm en visual studio
#1. Desde Google escribir node.js
#2. Instalar la versión más recomendada 
! npm install localtunnel


added 22 packages in 3s

3 packages are looking for funding
  run `npm fund` for details


npm notice
npm notice New minor version of npm available! 10.8.2 -> 10.9.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.9.0
npm notice To update run: npm install -g npm@10.9.0
npm notice


In [4]:
%%writefile app.py 
#Creamos el archivo de la APP en el interprete principal (Phyton)

# Archivo de la APP para el Dashboard de análisis de Barcelona

############################# IMPLEMENTACIÓN DE DASHBOARD ################################

# Importamos las librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from funpymodeling.exploratory import freq_tbl 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

#################################################################

# Cargar el dataset de Barcelona
@st.cache_resource
def load_data():
    # Lectura del archivo CSV sin índice y eliminamos la columna 'Unnamed: 0'
    df = pd.read_csv("Barcelona_limpios_valores.csv").drop(columns=['Unnamed: 0'], errors='ignore')
    
    # Selección de columnas categóricas y numéricas
    categorical_cols = df.select_dtypes(include=['object']).columns
    numeric_cols = df.select_dtypes(include=['float', 'int']).columns

    # Filtro univariado para 'property_type', 'room_type', y 'host_response_time'
    table = freq_tbl(df['property_type'])
    table1 = freq_tbl(df['room_type'])
    table2 = freq_tbl(df['host_response_time'])
    
    Filtro = table[table['frequency'] > 200]
    Filtro1 = table1[table1['frequency'] > 1]
    Filtro2 = table2[table2['frequency'] > 400]  

    # Ajusto el índice de mi dataframe
    Filtro_index = Filtro.set_index('property_type')        
    Filtro_index1 = Filtro1.set_index('room_type')               
    Filtro_index2 = Filtro2.set_index('host_response_time')

    # Devuelvo las tablas filtradas y las columnas categóricas y numéricas
    return Filtro_index, Filtro_index1, Filtro_index2, df, categorical_cols, numeric_cols

# Llamada a la función para cargar los datos
Filtro_index, Filtro_index1, Filtro_index2, df, categorical_cols, numeric_cols = load_data()

###################### CREACIÓN DEL DASHBOARD ################################

# 1. CREACIÓN DE LA SIDEBAR
st.sidebar.title("Airbnb Barcelona")
st.sidebar.header("Sidebar")
st.sidebar.subheader("Panel de selección")

# 2. CREACIÓN DE LOS FRAMES
Frames = st.selectbox(label="Frames", options=["Frame 1", "Frame 2", "Frame 3", "Frame 4","Frame 5","Frame 6"])

######################################################################################
# CONTENIDO DEL FRAME 1
if Frames == "Frame 1":
    st.title("Análisis Univariado - Barcelona")
    st.header("Gráfico de pastel")

    # Checkbox para mostrar el dataset completo
    check_box = st.sidebar.checkbox(label="Mostrar Dataset Completo")
    if check_box:
        st.write(df)
    
    # Widget para seleccionar entre las tres variables categóricas
    category_selected = st.sidebar.selectbox("Seleccionar categoría para gráfico de pastel", options=["property_type", "room_type", "host_response_time"])

    # Opción de mostrar frecuencia o porcentaje
    display_type = st.sidebar.radio("Mostrar como:", options=["Frecuencia", "Porcentaje"])

    # Selección de la tabla correspondiente
    if category_selected == "property_type":
        data_to_plot = Filtro_index
    elif category_selected == "room_type":
        data_to_plot = Filtro_index1
    else:
        data_to_plot = Filtro_index2

    # Si se elige mostrar en porcentaje, calcular porcentaje
    if display_type == "Porcentaje":
        total = data_to_plot['frequency'].sum()
        data_to_plot['percentage'] = (data_to_plot['frequency'] / total) * 100
        values_column = 'percentage'
    else:
        values_column = 'frequency'

    # Crear gráfico de pastel
    figure1 = px.pie(data_frame=data_to_plot, names=data_to_plot.index, values=values_column,
                     title=f"Distribución de {category_selected} ({display_type})", 
                     width=1600, height=600, color_discrete_sequence=px.colors.sequential.Reds)

    st.plotly_chart(figure1)


###############################################################################################
# CONTENIDO DEL FRAME 2
if Frames == "Frame 2":
    st.title("Análisis de Datos - Barcelona")
    st.header("Gráficos: Bar Plot e Histograma")

    # Seleccionar cualquier variable categórica del dataset para el Bar Plot
    variable_selected = st.sidebar.selectbox("Seleccionar variable categórica para el Bar Plot", options=categorical_cols)

    # Selección entre Frecuencia o Porcentaje para ambos gráficos
    tipo_grafico = st.sidebar.selectbox("Seleccionar tipo de gráfico para ambos gráficos", options=["Frecuencia", "Porcentaje"])

    # Selección de variable numérica para el Histograma
    num_var_selected = st.sidebar.selectbox("Seleccionar variable numérica para el Histograma", options=numeric_cols)

    # Generar la tabla de frecuencia como en el Frame 1
    freq_table = freq_tbl(df[variable_selected])

    if tipo_grafico == "Porcentaje":
        freq_table['porcentaje'] = (freq_table['frequency'] / freq_table['frequency'].sum()) * 100
        figure_bar = px.bar(data_frame=freq_table, x=variable_selected, y='porcentaje', 
                            title=f"Bar Plot de {variable_selected} (Porcentaje)", 
                            width=1600, height=600, color_discrete_sequence=['#FF0000'],
                            labels={'porcentaje': 'Porcentaje'})

        # Para el histograma en porcentaje
        df['porcentaje'] = (df[num_var_selected] / df[num_var_selected].sum()) * 100
        figure_hist = px.histogram(df, x='porcentaje', 
                                   title=f"Histograma de {num_var_selected} (Porcentaje)", 
                                   width=1600, height=600, color_discrete_sequence=['#FF0000'],
                                   labels={'porcentaje': 'Porcentaje'})
    else:
        figure_bar = px.bar(data_frame=freq_table, x=variable_selected, y='frequency', 
                            title=f"Bar Plot de {variable_selected} (Frecuencia)", 
                            width=1600, height=600, color_discrete_sequence=['#FF0000'],
                            labels={'frequency': 'Frecuencia'})

        # Histograma en frecuencia
        figure_hist = px.histogram(df, x=num_var_selected, 
                                   title=f"Histograma de {num_var_selected} (Frecuencia)", 
                                   width=1600, height=600, color_discrete_sequence=['#FF0000'],
                                   labels={num_var_selected: 'Frecuencia'})

    # Mostrar ambos gráficos uno debajo del otro
    st.plotly_chart(figure_bar)
    st.plotly_chart(figure_hist)

####################################################################################################################
#CONTENIDO DEL FRAME 3
if Frames == "Frame 3":
    st.title("Análisis de Datos - Barcelona")
    st.header("Dispersiones")

    # Widget para seleccionar cualquier variable categórica y numérica
    x_selected = st.sidebar.selectbox("Seleccionar variable categórica para el eje X", options=categorical_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable numérica para el eje Y", options=numeric_cols)

    # GRAPH: Scatter Plot 
    figure3 = px.scatter(df, x=x_selected, y=y_selected, title=f"Dispersión entre {x_selected} vs {y_selected}",width=1600, height=600, color_discrete_sequence=['#FF0000'])
    
    st.plotly_chart(figure3)

####################################################################################################################
# CONTENIDO DEL FRAME 4 (Regresión No Lineal)
if Frames == "Frame 4":
    st.title("Análisis de Regresión No Lineal - Barcelona")

    # Selección de variables independientes
    x_selected = st.sidebar.selectbox("Seleccionar variable independiente (X)", options=numeric_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable dependiente (Y)", options=numeric_cols)

    # Preparar los datos
    X = df[[x_selected]].dropna()  # Eliminar filas con NaN
    y = df[y_selected].loc[X.index]  # Asegurar que `y` tenga las mismas filas que `X`

    # Selección del tipo de regresión no lineal
    regression_type = st.sidebar.selectbox("Seleccionar tipo de función para la regresión no lineal", 
                                           options=["Polinómica (grado 2)", "Exponencial", "Logarítmica", "Senoidal"])

    # Aplicar diferentes tipos de regresión no lineal
    if regression_type == "Polinómica (grado 2)":
        poly = PolynomialFeatures(degree=2)
        X_transformed = poly.fit_transform(X)
    elif regression_type == "Exponencial":
        X_transformed = np.exp(X)
    elif regression_type == "Logarítmica":
        X_transformed = np.log(X + 1)  # Evitar log(0)
    elif regression_type == "Senoidal":
        X_transformed = np.sin(X)

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # Crear el modelo de regresión y entrenarlo
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Mostrar los resultados de la regresión
    st.write(f"Función seleccionada: {regression_type}")
    st.write(f"Coeficientes del modelo: {model.coef_}")
    st.write(f"R²: {r2_score(y_test, y_pred)}")

    # Mostrar el scatter plot de los valores predichos frente a los valores reales con colores rojo y amarillo
    figure4 = px.scatter(x=y_test, y=y_pred, labels={'x': 'Valores Reales', 'y': 'Valores Predichos'}, 
                         title=f"Regresión No Lineal ({regression_type}): Valores Reales vs Predichos", 
                         width=1600, height=600)

    # Añadir distinción de valores reales y predichos
    figure4.add_scatter(x=y_test, y=y_test, mode='markers', name='Valores Reales', 
                        marker=dict(color='rgba(255, 0, 0, 1)', size=8))  # Color rojo para reales
    figure4.add_scatter(x=y_test, y=y_pred, mode='markers', name='Valores Predichos', 
                        marker=dict(color='rgba(255, 255, 0, 1)', size=8))  # Color amarillo para predichos

    st.plotly_chart(figure4)

####################################################################################################################
# CONTENIDO DEL FRAME 5 (REGRESIÓN LOGÍSTICA)
if Frames == "Frame 5":
    st.title("Regresión Logística - Barcelona")
    
    # Eliminar los valores "Unknown" de la variable 'host_is_superhost'
    df = df[df['host_is_superhost'] != "Unknown"]

    # Categorización de 'price' en "Barato" y "Caro"
    Max_price = df['price'].max()
    Min_price = df['price'].min()
    intervalos_price = np.linspace(Min_price, Max_price, 3)  # Dividimos en 2 categorías
    categorias_price = ["Barato", "Caro"]
    df['Precios'] = pd.cut(df['price'], bins=intervalos_price, labels=categorias_price)

    # Categorización de 'accommodates' en "Incómodo" y "Cómodo"
    Max_accommodates = df['accommodates'].max()
    Min_accommodates = df['accommodates'].min()
    intervalos_accommodates = np.linspace(Min_accommodates, Max_accommodates, 3)  # Dos categorías
    categorias_accommodates = ["Incómodo", "Cómodo"]
    df['Comodidad'] = pd.cut(df['accommodates'], bins=intervalos_accommodates, labels=categorias_accommodates)

    # Categorización de 'number_of_reviews' en "Pocas" y "Muchas"
    Max_reviews = df['number_of_reviews'].max()
    Min_reviews = df['number_of_reviews'].min()
    intervalos_reviews = np.linspace(Min_reviews, Max_reviews, 3)  # Dos categorías
    categorias_reviews = ["Pocas", "Muchas"]
    df['Reseñas'] = pd.cut(df['number_of_reviews'], bins=intervalos_reviews, labels=categorias_reviews)

    # Categorización de 'review_scores_rating' en "Muy Mala" y "Muy Buena"
    Max_rating = df['review_scores_rating'].max()
    Min_rating = df['review_scores_rating'].min()
    intervalos_rating = np.linspace(Min_rating, Max_rating, 3)  # Dos categorías
    categorias_rating = ["Muy Mala", "Muy Buena"]
    df['Calificación'] = pd.cut(df['review_scores_rating'], bins=intervalos_rating, labels=categorias_rating)

    # Lista de variables dicotómicas y categorizadas
    transformed_vars = ['Precios', 'Comodidad', 'host_is_superhost', 'Reseñas', 'Calificación']

    # Convertir variables categóricas a numéricas utilizando LabelEncoder
    le = LabelEncoder()
    for var in transformed_vars:
        df[var] = le.fit_transform(df[var])

    # Selección de variables independientes (solo numéricas) y dependientes
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable categórica dependiente (Y)", options=transformed_vars)

    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Preparar los datos para la regresión logística
        X = df[x_selected_multiple]  # Variables independientes ya seleccionadas
        y = df[y_selected]  # Variable dependiente

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

        # Estandarización de los datos
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        # Definir el algoritmo de regresión logística
        algoritmo = LogisticRegression(max_iter=1000)

        # Entrenar el modelo
        algoritmo.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = algoritmo.predict(X_test)

        # Calcular y mostrar las métricas del modelo
        exactitud = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="macro")  # Cambiado a 'macro' para multiclase
        sensibilidad = recall_score(y_test, y_pred, average="macro")  # Cambiado a 'macro' para multiclase

        st.write(f"Exactitud del modelo: {exactitud:.2f}")
        st.write(f"Precisión del modelo: {precision:.2f}")
        st.write(f"Sensibilidad del modelo: {sensibilidad:.2f}")

        # Mostrar la matriz de confusión
        matriz = confusion_matrix(y_test, y_pred)
        st.write("Matriz de Confusión:")
        st.write(matriz)


############################################################################################
# CONTENIDO DEL FRAME 6 (REGRESIÓN LINEAL MÚLTIPLE)
if Frames == "Frame 6":
    st.title("Regresión Lineal Múltiple - Barcelona")

    # Ingresar variables a utilizar en la regresión
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols)
    y_selected = st.sidebar.selectbox("Seleccionar variable dependiente (Y)", options=numeric_cols)

    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Preparar los datos
        X = df[x_selected_multiple]  # Variables independientes seleccionadas
        y = df[y_selected].values.reshape(-1, 1)  # Variable dependiente seleccionada (reshape para hacerla 2D)

        # Estandarización de los datos (Incluyendo la variable dependiente)
        escalar_X = StandardScaler()
        escalar_y = StandardScaler()

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        X_train = escalar_X.fit_transform(X_train)
        X_test = escalar_X.transform(X_test)
        y_train = escalar_y.fit_transform(y_train)
        y_test = escalar_y.transform(y_test)

        # Definir el modelo de regresión lineal múltiple
        modelo = LinearRegression()

        # Entrenar el modelo
        modelo.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = modelo.predict(X_test)

        # Calcular las métricas del modelo
        from scipy.stats import pearsonr
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Calcular el coeficiente de correlación de Pearson
        correlation, _ = pearsonr(y_test.flatten(), y_pred.flatten())

        # Mostrar resultados en Streamlit
        st.write(f"Coeficiente de determinación (R²): {r2:.2f}")
        st.write(f"Coeficiente de correlación de Pearson: {correlation:.2f}")

        # Mostrar coeficientes del modelo
        st.write("Coeficientes del modelo:")
        coef_df = pd.DataFrame({"Variable": x_selected_multiple, "Coeficiente": modelo.coef_.flatten()})
        st.write(coef_df)

        # Mostrar el modelo matemático
        coef_list = [f"{coef:.4f} * {var}" for coef, var in zip(modelo.coef_.flatten(), x_selected_multiple)]
        modelo_mat = " + ".join(coef_list)
        st.write(f"Modelo matemático: Predicción = {modelo.intercept_[0]:.4f} + {modelo_mat}")

        # Asignar colores distintos a cada variable independiente seleccionada
        colors = plt.cm.get_cmap('tab10', len(x_selected_multiple))

        # Mostrar gráfico de predicciones vs valores reales (con diferentes colores para cada variable)
        fig, ax = plt.subplots()
        colors = plt.cm.Reds(np.linspace(0.4, 1, len(x_selected_multiple)))  # Genera tonos de rojo

        for i, var in enumerate(x_selected_multiple):
            ax.scatter(y_test, X_test[:, i], color=colors[i], label=f"Predicción {var}")  # Predicciones por variable

        ax.scatter(y_test, y_test, color='yellow', label='Valores Reales', alpha=0.5)  # Valores reales en amarillo
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Línea de referencia
        ax.set_xlabel('Valores Reales')
        ax.set_ylabel('Predicciones')
        ax.legend()
        st.pyplot(fig)

        # Opción de mostrar un heatmap de correlaciones
        if st.checkbox("Mostrar Heatmap de Correlaciones"):
            # Calcular la matriz de correlaciones
            corr_matrix = df[[y_selected] + x_selected_multiple].corr()

            # Mostrar el heatmap
            fig_heatmap, ax_heatmap = plt.subplots()
            sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=ax_heatmap)
            st.pyplot(fig_heatmap)


Overwriting app.py
